**Note:** for running the example we need at least a `ml.m5.large` instance

# SETUP

## Check the version of python using

In [2]:
!which python3

## Install required libraries

In [3]:
!pip install --upgrade pip

In [4]:
!pip install ipywidgets --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.0 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.0 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.14.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a6 which is incompatible.
spyder-kernels 2.3.3 requires ipython<8,>=7.31.1; python_version >= "3", but you have ipython 8.14.0 which is incompatible.


In [5]:
!pip install boto3 --quiet
!pip install langchain --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.14.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a6 which is incompatible.


In [6]:
!pip install torch --quiet

In [7]:
!pip install sentence_transformers --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.4 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.


In [8]:
!pip install chromadb --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.1 which is incompatible.


In [9]:
!pip install typing-extensions --upgrade

In [10]:
!pip install SQLAlchemy==2.0.1 --quiet
!pip install flask-sqlalchemy --quiet



In [11]:
!pip install unstructured --quiet

In [12]:
!pip install pdf2image
!pip install pdfminer
!pip install pdfminer.six

  Using cached pdf2image-1.16.3-py3-none-any.whl (11 kB)
  Using cached pdfminer-20191125-py3-none-any.whl
  Obtaining dependency information for pycryptodome from https://files.pythonhosted.org/packages/40/88/89623388754162ddb82c62814ccbfdbfcc8ed9bd6d5f7412d2479bdca3a7/pycryptodome-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pycryptodome-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached pycryptodome-3.18.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)


In [13]:
!pip install rouge

  Using cached rouge-1.0.1-py3-none-any.whl (13 kB)


In [14]:
!pip3 install opencv-python-headless --quiet #required for cv2

## Import libraries

In [15]:
import chromadb 

In [16]:
import sentence_transformers

In [17]:
import langchain

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import NLTKTextSplitter
import os

### ContentHandler for LLama-2-chat

In [19]:
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs" : [[{"role" : "system",
        "content" : "Your are a kind assistant for the supreme court. All your answers MUST be in grammatically correct spanish. NEVER. Under any circunstance you have to include words in english. if you that you will be fired."},
        {"role" : "user", "content" : prompt}]],
        "parameters" : {**model_kwargs}})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generation"]["content"]

### ContentHandler for FALCON

In [20]:
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json
class HFContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs) -> bytes:
        self.len_prompt = len(prompt)
        input_dict = {
            "inputs": prompt,
            "parameters": model_kwargs
        }
        input_str = json.dumps(input_dict)
        #print(input_str)
        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> str:
        response_json = output.read()
        res = json.loads(response_json)
       # print(res)

        # stripping away the input prompt from the returned response
        ans = res[0]['generated_text'][self.len_prompt:]
        ans = ans[:ans.rfind("Human")].strip()
        return ans

## Setup the llm 

In [97]:
import boto3
region = boto3.Session().region_name
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-7b-f"
#endpoint_name  = "jumpstart-dft-hf-llm-falcon-7b-instruct-bf16"
content_handler = ContentHandler()

llm=SagemakerEndpoint(
     endpoint_name=endpoint_name, 
     region_name=region, 
     model_kwargs={"max_new_tokens": 1000, "top_p": 0.9, "temperature": 0.01},
     endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
     content_handler=content_handler
 )


### Test the LLM

In [98]:
print(llm("Deci algo en español"))

 ¡Por supuesto, señor/a! Como asistente leal del Tribunal Supremo, estoy a su disposición para ayudarle en cualquier necesidad. ¿En qué puedo ser de ayuda?


## Setup SBERT for calculating embeddings

In [27]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [28]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

# INTERACTIVE EXAMPLE

## Load a PDF and calculate the embeddings

In [30]:
loader = UnstructuredPDFLoader("/root/JurisGPT/rawdata/laboral/10000003368.pdf")

In [31]:
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])

In [24]:
index.vectorstore.delete_collection()

## Ask LLM to summarize each section

In [32]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Antecedentes' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_ant = (index.query(query,llm=llm))
print(summary_ant)

In [217]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_1 = (index.query(query,llm=llm))
print(summary_1)


In [218]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la segunda cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_2 = (index.query(query,llm=llm))
print(summary_2)

In [235]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la tercera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_3 = (index.query(query,llm=llm))
print(summary_3)

In [220]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Resuelve' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_res = (index.query(query,llm=llm))
print(summary_res)

In [221]:
prompt = f"Eres un miembro de la corte suprena de Mendoza en Argentina. A partir del siguiente resumen de una setencia de la corte :\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
prompt +="Explicar las causas detras de la decision final de la sentencia. Escribe un sumario de la corte en no menos de 250 palabras. Favor de no mencionar:\n1. Los nombres de las secciones.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n\n"
prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia." 
print(prompt)

In [222]:
print(llm(prompt,))

# FUNCTIONS

## Create summary (ES)

In [93]:
def create_summary_es(rule_file, embedding, llm):
    loader = UnstructuredPDFLoader(rule_file)
    #index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    #index.vectorstore.delete_collection()
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    print("calculating ant")
    query = "Eres un secretario de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Antecedentes' de una sentencia de la corte. Escribe tu respuesta en no mas de 150 palabras:"
    summary_ant = (index.query(query,llm=llm))
    print("calculating 1")
    query = "Eres un secretario de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. Escribe tu respuesta en no mas de 150 palabras:"
    summary_1 = (index.query(query,llm=llm))
    print("calculating 2")
    query = "Eres un secretario de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Sobre la segunda cuestion' de una sentencia de la corte. Escribe tu respuesta en no mas de 150 palabras:"
    summary_2 = (index.query(query,llm=llm))
    print("calculating 3")
    query = "Eres un secretario de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Sobre la tercera cuestion' de una sentencia de la corte. Escribe tu respuesta en no mas de 150 palabras:"
    summary_3 = (index.query(query,llm=llm))
    print("calculating res")
    query = "Eres un secreatrio de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Resuelve' de una sentencia de la corte. Escribe tu respuesta en no mas de 150 palabras:"
    summary_res = (index.query(query,llm=llm))
    
    prompt = f"Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente resumen de una sentencia de la corte :\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"    
    prompt +="Presentar un sumario en donde se expliquen las causas detras de la decision final de la sentencia. Escribir el sumario de la corte en no mas de 700 palabras."
    prompt +="Favor de no mencionar NUNCA:\n1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n5. La palabra corte\n\n"
    prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia. En caso de disidencia indicar el nombre del  ministro de la corte. " 
    #print(prompt)
    index.vectorstore.delete_collection()
    print("calculating prompt")
    #print(prompt)
    res = llm(prompt)
    return res

In [ ]:
prompt+=""" 
    
Sabiendo que un sumario es una descripción resumida de las doctrinas contempladas en la sentencia judicial dado que en ella pueden tratarse diversas cuestiones jurídicas       
Presentar un sumario en donde se expliquen las causas detras de la decision final de la sentencia. Escribir el sumario de la corte en no mas de 700 palabras."

En el sumario incluir:

1. Indicar que eventos o circunstancias son centrales para el caso?
2. Indicar cómo se valoran las conductas o eventos en el contexto laboral (subjetiva y objetivamente)?
3. Indicar como afectan estos hechos la relación entre las partes involucradas (empleador y empleado)?
4. Indicar leyes, principios o derechos están en juego (tutela sindical, deber de fidelidad, igualdad de oportunidades, etc.)?
5. Indicar qué determinación se hizo respecto a la legalidad de la conducta o evento en cuestión
6. Indicar si hubo disidencia entre los jueces

En el sumario no mencionar NUNCA:

1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.
2. Los nombres de los casos.
3. Los nombres de los ministros de la corte.
4. Las fechas y la ubicacion de la corte.
5. La palabra corte
6. Los nombres de las personas involucradas.
Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia.

Limitarse a generar el sumario. No responder Por supuesto.
""" 

## Create Summary (EN)

In [16]:
def create_summary(rule_file, embedding, llm):
    loader = UnstructuredPDFLoader(rule_file)
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    index.vectorstorae.delete_collection()
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Antecedentes' of a particular court ruling . Write your answer in spanish in no less than 150 words:"
    summary_ant = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la primera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_1 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la segunda cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_2 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la tercera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_3 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'resuelve' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_res = (index.query(query,llm=llm))

    prompt = f"You are a member of the supreme court, Argentina. Given the following summary of a particular court ruling:\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
    prompt +="Write a court rule summary in spanish in no less than 150 words. Do not mention:\n1. the name of the the sections.\n2. the numbers of the cases.\n3. the names of the ministers of the court.\n4. dates and court locations..\n\n"
    prompt +="When writing the summary keep in mind it can help for future jurisprudence. Explain the causes behind the final decision. Translate your answer to spanish:"
    index.vectorstore.delete_collection()
    return llm(prompt)


In [86]:
file_name = "/root/JurisGPT/rawdata/laboral/10000003247.pdf"
summary = create_summary_es(file_name, embedding = embedding, llm = llm)

calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente resumen de una sentencia de la corte :

```
 ¡Por supuesto! Aquí está el resumen de la sección "Antecedentes" de la sentencia de la Corte Suprema de Mendoza en Argentina, en español y con un límite de 150 palabras:

En la causa N° 13-04795867-3/1, la Sala Segunda de la Excma. Suprema Corte de Justicia de Mendoza tomó en consideración la sentencia definitiva sobre el recurso extraordinario interpuesto por FINAMED S.A. contra la resolución de la Cámara de Apelaciones de Mendoza, que confirmó la sentencia de primera instancia que declaró la exclusión de tutela sindical de la trabajadora Nancy Elizabeth Rodríguez. Los señores ministros del tribunal establecieron el orden de estudio de la causa y se estableció que la sentencia es arbitraria por no tener en consideración las circunstancias del caso y por no eva

In [87]:
print(summary)

 ¡Por supuesto! Aquí está el sumario de la sentencia de la Corte Suprema de Mendoza en Argentina, en español y con un límite de 700 palabras:

El caso se refiere a la exclusión de tutela sindical de la trabajadora Nancy Elizabeth Rodríguez, quien fue despedida por su empleador, FINAMED S.A., por motivos de pérdida de confianza y falta de fidelidad. La defensa argumentó que la sentencia fue arbitraria al no tener en consideración las circunstancias del caso y la valoración de la injuria.

El tribunal evaluó los hechos y elementos probatorios en forma absurda, no teniendo en cuenta las circunstancias del caso, lo que fue controvertido por la defensa. Se evaluó la causal invocada y se consideró la pérdida de confianza como causal de despido. La Corte falló en definitiva y resolvió que el recurso extraordinario provincial interpuesto a fs. 07/25 no prospera.

Los hechos centrales del caso son la pérdida de confianza y la falta de fidelidad, que se evaluaron en el contexto laboral y se cons

In [108]:
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
name, extension = os.path.splitext(os.path.basename(file_name))
summary_filename = f"{name}_summary.txt"
summary_file_path = os.path.join(output_directory_path, summary_filename)
with open(summary_file_path, 'w') as f:
    f.write(summary)


## Create summary from dir

In [33]:
def create_summary_from_dir(directory_path, output_directory_path, embedding, llm):
    import time
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        # Extract the file extension
        name, extension = os.path.splitext(filename)
        
        # Build the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Skip directories, only work on PDF files
        if os.path.isfile(file_path) and extension.lower() == '.pdf':
            print(f'working on {file_path}')
            # Generate summary
            start_time = time.time()
            summary = create_summary_es(file_path, embedding, llm)
            stop_time = time.time()
            
            # Save summary to new file
            summary_filename = f"{name}_summary.txt"
            summary_file_path = os.path.join(output_directory_path, summary_filename)
            
            with open(summary_file_path, 'w') as f:
                f.write(summary)
            print(f'created {summary_file_path} in {stop_time - start_time} seconds')


## Calculate ROUGE metric

In [43]:
import os
import re
from rouge import Rouge

def calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern):
    # Initialize Rouge
    rouge = Rouge()
    
    # Compile the regular expressions for filename patterns
    system_re = re.compile(system_filename_pattern)
    model_re = re.compile(model_filename_pattern)
    
    # Initialize storage for scores
    all_scores = []

    # Loop through the files in the system summaries directory
    for system_filename in os.listdir(system_dir):
        match = system_re.match(system_filename)
        if not match:
            continue

        # Extract ID
        system_id = match.groups()[0]

        # Find the corresponding model summary file using ID
        model_filename = f"{system_id}_summary.txt"
        model_path = os.path.join(model_dir, model_filename)

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found. Attempted to open {model_path}")
            continue

        

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found.")
            continue

        # Read system and model summaries
        with open(os.path.join(system_dir, system_filename), 'r', encoding='utf-8') as f:
            system_summary = f.read().strip()
            system_summary = re.sub('<.*?>', '', system_summary)
            #print(system_summary)
        
        with open(model_path, 'r', encoding='utf-8') as f:
            model_summary = f.read().strip()
    
        # Compute the ROUGE score
        score = rouge.get_scores(system_summary, model_summary)[0]
        #all_scores.append(score)
        all_scores.append({
            'system_id': system_id,
            'rouge-1': score['rouge-1'],
            'rouge-2': score['rouge-2'],
            'rouge-l': score.get('rouge-l', {})  # Add this line to include ROUGE-L
        })
        # Compute the ROUGE score


        # Extract ROUGE-1, ROUGE-2, and ROUGE-3 scores
        #rouge_1 = score.get('rouge-1', {})
        #rouge_2 = score.get('rouge-2', {})
       
        # Pretty-print scores
        #print(f"ROUGE scores for ID {system_id}:")
        #print(f"  ROUGE-1: R: {rouge_1.get('r', 'N/A'):.4f}, P: {rouge_1.get('p', 'N/A'):.4f}, F: {rouge_1.get('f', 'N/A'):.4f}")
        #print(f"  ROUGE-2: R: {rouge_2.get('r', 'N/A'):.4f}, P: {rouge_2.get('p', 'N/A'):.4f}, F: {rouge_2.get('f', 'N/A'):.4f}")
      
        
        #print(f"ROUGE score for ID {system_id}: {score}")

    # If you want average scores, you can calculate that here from all_scores.
    # ...

    return all_scores


## Print ROUGE TABLE

In [35]:
import pandas as pd  # Don't forget to import pandas

def print_rouge_table(all_scores):
    pd.set_option('display.max_rows', None)
    
    system_ids = []
    metrics = []
    rs = []
    ps = []
    fs = []

    # Loop through each dictionary to extract the values
    for score_dict in all_scores:
        system_id = score_dict['system_id']

        for metric, values in score_dict.items():
            if metric == 'system_id':
                continue  # Skip the system_id, we've already saved it

            system_ids.append(system_id)
            metrics.append(metric)
            rs.append(values['r'])
            ps.append(values['p'])
            fs.append(values['f'])

    # Create the DataFrame
    df_long = pd.DataFrame({
        'system_id': system_ids,
        'metric': metrics,
        'r': rs,
        'p': ps,
        'f': fs
    })

    return df_long

# UTILS

## Stop LLAMA Jumpstart endpoint

In [115]:
import boto3

aws_region = boto3.Session().region_name
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-70b-f"

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c3bf5110-22d1-43df-8c5b-c612bb5bc5fc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c3bf5110-22d1-43df-8c5b-c612bb5bc5fc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 02 Sep 2023 03:23:56 GMT'},
  'RetryAttempts': 0}}

# MAIN

## Create Summaries

In [99]:
directory_path = "/root/JurisGPT/rawdata/laboral/"  
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
create_summary_from_dir(directory_path, output_directory_path, embedding = embedding, llm = llm)

working on /root/JurisGPT/rawdata/laboral/10000003220.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003220_summary.txt in 52.93086791038513 seconds
working on /root/JurisGPT/rawdata/laboral/10000003219.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003219_summary.txt in 66.93007588386536 seconds
working on /root/JurisGPT/rawdata/laboral/10000003275.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003275_summary.txt in 68.05711078643799 seconds
working on /root/JurisGPT/rawdata/laboral/10000003307.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003307_summary.txt in 70.96445655822754 seconds
working on /root/JurisGPT/ra

## Calculate ROUGE

The ROUGE-1 metric is a type of Recall-Oriented Understudy for Gisting Evaluation (ROUGE) score that is commonly used for evaluating the quality of summaries by comparing them to reference summaries. Specifically, ROUGE-1 measures the overlap of unigrams (single words) between the generated summary and the reference summary.

The ROUGE-1 score comprises three values:

1. **Precision (P)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the generated summary. In essence, precision answers the question: "Of all the unigrams in the generated summary, how many are in the reference summary?"
   \[
   \text{Precision} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in generated summary}}
   \]

2. **Recall (R)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the reference summary. Recall answers the question: "Of all the unigrams in the reference summary, how many are in the generated summary?"
   \[
   \text{Recall} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in reference summary}}
   \]

3. **F1 Score (F)**: This is the harmonic mean of precision and recall and serves as a single number that balances the trade-off between precision and recall.
   \[
   \text{F1 Score} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
   \]

ROUGE-1 is particularly useful for assessing the quality of extractive summaries, where sentences or phrases are selected from the original text to form a summary. The metric can also be used for abstractive summaries but may not capture the quality of the summary as effectively as when combined with other ROUGE metrics like ROUGE-2 (bigram overlap) or ROUGE-L (longest common subsequence).

# 7B Results

In [112]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/7b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [114]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.245455,0.221311,0.232759
3,10000003307,rouge-1,0.319277,0.162577,0.215447
6,10000003220,rouge-1,0.179856,0.193798,0.186567
9,10000003219,rouge-1,0.276730,0.129412,0.176353
12,10000003258,rouge-1,0.327381,0.211538,0.257009
15,10000003300,rouge-1,0.322404,0.243802,0.277647
18,10000003368,rouge-1,0.227027,0.214286,0.220472
21,10000003221,rouge-1,0.349315,0.262887,0.300000
24,10000003292,rouge-1,0.229665,0.149533,0.181132
27,10000003247,rouge-1,0.262195,0.210784,0.233696


In [115]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.27393049419520055, 0.05468252084765027)

# 13B results

In [104]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/13b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [105]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.283688,0.163934,0.207792
3,10000003307,rouge-1,0.352459,0.131902,0.191964
6,10000003220,rouge-1,0.141935,0.170543,0.154930
9,10000003219,rouge-1,0.314050,0.111765,0.164859
12,10000003258,rouge-1,0.281879,0.161538,0.205379
15,10000003300,rouge-1,0.379032,0.194215,0.256831
18,10000003368,rouge-1,0.230216,0.163265,0.191045
21,10000003221,rouge-1,0.315789,0.247423,0.277457
24,10000003292,rouge-1,0.299320,0.137072,0.188034
27,10000003247,rouge-1,0.286765,0.191176,0.229412


In [111]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.2885133217794919, 0.06561640326263223)